# Valeur foncière moyenne : définition d'un loyer au m²



In [4]:
#import all the libraries needed
import statistics
import requests 
import json
import math

In [1]:
rayon = 50
lat = 48.85 #latitude
lon = 2.35 #longitude

def valeur_fonciere_au_m2(rayon, lat, lon):

    params = {"format": "json"}
    response = requests.get(f"https://api.cquest.org/dvf?lat={lat}&lon={lon}&dist={rayon}", params=params) #requête de l'API

    def jprint(obj):
        # création d'un string à partir de l'objet JSON
        text = json.dumps(obj, indent=4)
        print(text)

    valeur_fonciere_moyenne = [] # tableau où on stocke les valeurs foncières se trouvant dans le périmètre choisi

    valeur_api = response.json() #la valeur foncière que l'on cherche avec l'API

    for i in range(len(valeur_api["features"])): 
        surface = (valeur_api["features"][i]["properties"]["surface_relle_bati"]) #Recherche de surface du logement
        if surface != 0 :
            prix = (valeur_api["features"][i]["properties"]["valeur_fonciere"]) #Recherche de la valeur foncière
            prix_au_m2 = prix/surface
            valeur_fonciere_moyenne.append(prix_au_m2)

    return round(statistics.mean(valeur_fonciere_moyenne))

In [3]:
#print(f"La valeur foncière moyenne sur un rayon de {rayon}m est de {valeur_fonciere_au_m2(rayon, lat, lon)} euros par m2.\n")

#  Coûts de construction

### Estimation peu fiable

In [4]:
def cout_construction_immeuble1(surface, lon, lat): 
    assert(surface >= 0)

    surface += surface/10 #surface totale = surface logement + partie commune ≈ 10%

    prix_loyer = valeur_fonciere_au_m2(20, lon, lat)*surface #Exemple rayon de 20m pour le calcul de valeur_fonciere_au_m2
    
    prix_etudes_de_sol = prix_loyer*0.02
    prix_controles_techniques = prix_loyer*0.03
    prix_honoraires_architectes = prix_loyer*0.08
    return round(prix_loyer + prix_etudes_de_sol + prix_controles_techniques + prix_honoraires_architectes)

### Estimation plus fiable 

# Coûts de destruction 


In [14]:
def calculer_cout_demolition(surface, nombre_etages, indice_complexite, indice_etat): #les indices sont entre 1-3
    # Coûts initiaux par unité de surface
    cout_par_metre_carre = 100  # Coût de base par mètre carré
    
    # Facteurs multiplicateurs en fonction de la complexité et de l'état du bâtiment
    facteur_complexite = [1, 1.2, 1.5]  # Indice de complexité (1-3)
    facteur_etat = [1, 1.1, 1.3]  # Indice d'état (1-3)
    
    # Calcul des coûts de base
    cout_base = surface * cout_par_metre_carre
    
    # Calcul des coûts en fonction de la complexité et de l'état du bâtiment
    cout_total = cout_base * facteur_complexite[indice_complexite-1] * facteur_etat[indice_etat-1]
    
    # Coûts supplémentaires en fonction du nombre d'étages
    cout_supplementaire_etages = 5000 * nombre_etages
    
    # Ajout des coûts supplémentaires
    cout_total += cout_supplementaire_etages
    
    return round(cout_total)


In [22]:
# Demande des informations à l'utilisateur

surface = 200     #surface du bâtiment cumulée  (en mètres carrés)
nombre_etages = 2      # nombre d'étages du bâtiment)
indice_complexite = 2  #indice de complexité du bâtiment (de indice_standing à 3)
indice_etat = 1        #indice de l'état du bâtiment (de indice_standing à 3)

# Appel de la fonction pour calculer les coûts de démolition
cout_demolition = calculer_cout_demolition(surface, nombre_etages, indice_complexite, indice_etat)

# Affichage du résultat
print("Le coût de démolition du bâtiment est de", cout_demolition, "euros.")

Le coût de démolition du bâtiment est de 32080 euros.


# Coûts de réhabilitation 

### Définition des données dans un dictionnaire 

In [4]:
import math

# Problèmes de construction
geotechnical_investigation_prices = {
    "étude topographique": 2.16,  # € par mètre carré
    "rapport ingénieur": 2100  # €
}

demolition_prices = {
    "lavabo": 51,  # € chacun
    "baignoire": 82,  # € chacune
    "douche": 137,  # € chacune
    "lampe incandescente": 17,  # € chacune
    "lave-vaisselle": 31,  # € chacun
    "porte": 55,  # € chacune
    "fenêtre": 56,  # € chacune
    "poutre": 16.40,  # € par mètre linéaire
    "cloison en plâtre": 4.95,  # € par mètre carré
    "charpente": 11.08,  # € par mètre linéaire
    "tuiles": 15.50,  # € par mètre carré
    "murs": 43.92,  # € par mètre carré
    "murs porteurs": 174.04,  # € par mètre carré
    "sol en béton": 48.70,  # € par mètre carré
    "location camion hydraulique 12T": 1400  # € par jour
}

# Dégats des eaux
mold_prices = {
    "inspection de la zone": 481,  # €
    "réduction de la moisissure": 1923,  # €
    "enlèvement de zones contaminées par la moisissure": 18132  # €
}

mold_demolition_prices = {
    "sol": 3.29,  # € par pied carré (10.79 € par mètre carré)
    "plafond": 2.17  # € par pied carré (7.12 € par mètre carré)
}

# Problèmes électriques
electrical_replacement_prices = {
    "remplacement du panneau électrique": (879, 2747.25),  # €
    "réparation ou remplacement des fils et des câbles": (546, 1638),  # €
    "installation de nouveaux circuits électriques": (546, 1638),  # €
    "installation de systèmes d'éclairage": (227, 1137)  # € par pièce
}

electrical_upgrade_prices = {
    # € par prise ou interrupteur
    "mise à niveau des prises de courant et des interrupteurs": (72.50, 108.75),
    "mise à jour des systèmes de sécurité": (546, 2185)  # €
}

# Problèmes de plomberie
bathroom_prices = {
    "lavabo": 60.06,  # €
    "baignoire/douche": 150.15,  # €
    "toilette": (200, 600),  # €
    "robinet de douche": (60, 120),  # €
    "robinet de lavabo": (30, 100)  # €
}

plumbing_repair_prices = {
    "réparation d'une fuite": (200, 550),  # €
    "remplacement d'un tuyau": (100, 200),  # €
    "remplacement d'un robinet": (100, 250),  # €
    "débouchage d'un évier": (100, 200),  # €
    "remplacement d'un évier": (200, 500)  # €
}

# Problème de chauffage, ventilation, climatisation
hvac_maintenance_prices = {
    "entretien annuel d'une chaudière": (120, 200),  # €
    "entretien du système électronique": 74.62,  # €
    "entretien d'une pompe à chaleur": (100, 300),  # €
    "nettoyage des conduits de ventilation": (200, 500)  # €
}

hvac_replacement_repair_prices = {
    "ventilation + conduits": 546,  # €
    "chaudière": [1500, 3000, 4000, 7000],  # €
    # €
    "installation d'un nouveau système de chauffage (pompe à chaleur)": (5000, 12000),
    "réparation ou remplacement de la climatisation": (1000, 5000),  # €
    # €
    "installation d'un système de ventilation mécanique contrôlée (VMC)": (1000, 4000),
    # €
    "installation d'un système de climatisation centralisée": (3000, 8000)
}

# Problème d'isolation
insulation_prices = {
    "isolation des combles": (20, 80),  # € par mètre carré
    "isolation des murs": (50, 150),  # € par mètre carré
    "isolation des planchers": (30, 80),  # € par mètre carré
    "installation de fenêtres isolantes": (300, 1000),  # € par fenêtre
    "isolation des portes": (50, 500)  # € par porte
}

# Présence de nuisibles
pest_control_prices = {
    "élimination de souris et de rats": (150, 500),  # €
    "élimination de punaises de lit": (200, 1000),  # €
    "élimination des guêpes et des nids de frelons": (60, 250),  # €
    "élimination des fourmis": (70, 200)  # €
}

pest_damage_repair_prices = {
    "réparation des dommages causés par les rongeurs": (150, 500),  # €
    "réparation des dommages causés par les insectes": (200, 1000),  # €
    "nettoyage et désinfection de la zone touchée": (50, 100) # € par mètre carré
}

# Problèmes d'humidité
humidity_detection_treatment_prices = {
    "diagnostic de l'humidité": (80, 150),  # €
    # € par mètre linéaire
    "traitement de l'humidité par injection": (100, 150),
    "traitement de l'humidité par ventilation": (1500, 5000),  # €
    "traitement de l'humidité par drainage": (1000, 5000)  # €
}

# Problèmes de toiture
roof_repair_prices = {
    "remplacement de tuiles": (2, 4),  # € par tuile
    "réparation de fuite": (200, 400),  # €
    "remplacement du revêtement de toit": (40, 100),  # € par mètre carré
    "nettoyage de la toiture": (200, 500)  # €
}

roof_replacement_prices = {
    "toiture en tuiles": (60, 150),  # € par mètre carré
    "toiture en ardoise": (80, 250),  # € par mètre carré
    "toiture en zinc": (120, 200),  # € par mètre carré
    "toiture en bac acier": (30, 100)  # € par mètre carré
}

# Problèmes de fondation
foundation_crack_repair_prices = {
    # € par fissure
    "réparation de fissures - injection de résine époxy": (100, 300),
    "ancrage de la fondation": (2000, 5000)  # €
}

foundation_subsidence_repair_prices = {
    "injection de mousse polyuréthane": (1000, 3000),  # €
    "remplacement des pieux": (3000, 7000)  # €
}

foundation_wall_repair_prices = {
    # € par mètre carré
    "rénovation de mur en blocs de béton": (100, 150),
    # € par mètre carré
    "réparation de murs de fondation en briques": (100, 250),
    "rénovation de mur en béton coulé": (300, 500)  # € par mètre carré
}
import math

# Problèmes de construction
geotechnical_investigation_prices = {
    "étude topographique": 2.16,  # € par mètre carré
    "rapport ingénieur": 2100  # €
}

demolition_prices = {
    "lavabo": 51,  # € chacun
    "baignoire": 82,  # € chacune
    "douche": 137,  # € chacune
    "lampe incandescente": 17,  # € chacune
    "lave-vaisselle": 31,  # € chacun
    "porte": 55,  # € chacune
    "fenêtre": 56,  # € chacune
    "poutre": 16.40,  # € par mètre linéaire
    "cloison en plâtre": 4.95,  # € par mètre carré
    "charpente": 11.08,  # € par mètre linéaire
    "tuiles": 15.50,  # € par mètre carré
    "murs": 43.92,  # € par mètre carré
    "murs porteurs": 174.04,  # € par mètre carré
    "sol en béton": 48.70,  # € par mètre carré
    "location camion hydraulique 12T": 1400  # € par jour
}

# Dégats des eaux
mold_prices = {
    "inspection de la zone": 481,  # €
    "réduction de la moisissure": 1923,  # €
    "enlèvement de zones contaminées par la moisissure": 18132  # €
}

mold_demolition_prices = {
    "sol": 3.29,  # € par pied carré (10.79 € par mètre carré)
    "plafond": 2.17  # € par pied carré (7.12 € par mètre carré)
}

# Problèmes électriques
electrical_replacement_prices = {
    "remplacement du panneau électrique": (879, 2747.25),  # €
    "réparation ou remplacement des fils et des câbles": (546, 1638),  # €
    "installation de nouveaux circuits électriques": (546, 1638),  # €
    "installation de systèmes d'éclairage": (227, 1137)  # € par pièce
}

electrical_upgrade_prices = {
    # € par prise ou interrupteur
    "mise à niveau des prises de courant et des interrupteurs": (72.50, 108.75),
    "mise à jour des systèmes de sécurité": (546, 2185)  # €
}

# Problèmes de plomberie
bathroom_prices = {
    "lavabo": 60.06,  # €
    "baignoire/douche": 150.15,  # €
    "toilette": (200, 600),  # €
    "robinet de douche": (60, 120),  # €
    "robinet de lavabo": (30, 100)  # €
}

plumbing_repair_prices = {
    "réparation d'une fuite": (200, 550),  # €
    "remplacement d'un tuyau": (100, 200),  # €
    "remplacement d'un robinet": (100, 250),  # €
    "débouchage d'un évier": (100, 200),  # €
    "remplacement d'un évier": (200, 500)  # €
}

# Problème de chauffage, ventilation, climatisation
hvac_maintenance_prices = {
    "entretien annuel d'une chaudière": (120, 200),  # €
    "entretien du système électronique": 74.62,  # €
    "entretien d'une pompe à chaleur": (100, 300),  # €
    "nettoyage des conduits de ventilation": (200, 500)  # €
}

hvac_replacement_repair_prices = {
    "ventilation + conduits": 546,  # €
    "chaudière": [1500, 3000, 4000, 7000],  # €
    # €
    "installation d'un nouveau système de chauffage (pompe à chaleur)": (5000, 12000),
    "réparation ou remplacement de la climatisation": (1000, 5000),  # €
    # €
    "installation d'un système de ventilation mécanique contrôlée (VMC)": (1000, 4000),
    # €
    "installation d'un système de climatisation centralisée": (3000, 8000)
}

# Problème d'isolation
insulation_prices = {
    "isolation des combles": (20, 80),  # € par mètre carré
    "isolation des murs": (50, 150),  # € par mètre carré
    "isolation des planchers": (30, 80),  # € par mètre carré
    "installation de fenêtres isolantes": (300, 1000),  # € par fenêtre
    "isolation des portes": (50, 500)  # € par porte
}

# Présence de nuisibles
pest_control_prices = {
    "élimination de souris et de rats": (150, 500),  # €
    "élimination de punaises de lit": (200, 1000),  # €
    "élimination des guêpes et des nids de frelons": (60, 250),  # €
    "élimination des fourmis": (70, 200)  # €
}

pest_damage_repair_prices = {
    "réparation des dommages causés par les rongeurs": (150, 500),  # €
    "réparation des dommages causés par les insectes": (200, 1000),  # €
    "nettoyage et désinfection de la zone touchée": (50, 100) # € par mètre carré
}

# Problèmes d'humidité
humidity_detection_treatment_prices = {
    "diagnostic de l'humidité": (80, 150),  # €
    # € par mètre linéaire
    "traitement de l'humidité par injection": (100, 150),
    "traitement de l'humidité par ventilation": (1500, 5000),  # €
    "traitement de l'humidité par drainage": (1000, 5000)  # €
}

# Problèmes de toiture
roof_repair_prices = {
    "remplacement de tuiles": (2, 4),  # € par tuile
    "réparation de fuite": (200, 400),  # €
    "remplacement du revêtement de toit": (40, 100),  # € par mètre carré
    "nettoyage de la toiture": (200, 500)  # €
}

roof_replacement_prices = {
    "toiture en tuiles": (60, 150),  # € par mètre carré
    "toiture en ardoise": (80, 250),  # € par mètre carré
    "toiture en zinc": (120, 200),  # € par mètre carré
    "toiture en bac acier": (30, 100)  # € par mètre carré
}

# Problèmes de fondation
foundation_crack_repair_prices = {
    # € par fissure
    "réparation de fissures - injection de résine époxy": (100, 300),
    "ancrage de la fondation": (2000, 5000)  # €
}

foundation_subsidence_repair_prices = {
    "injection de mousse polyuréthane": (1000, 3000),  # €
    "remplacement des pieux": (3000, 7000)  # €
}

foundation_wall_repair_prices = {
    # € par mètre carré
    "rénovation de mur en blocs de béton": (100, 150),
    # € par mètre carré
    "réparation de murs de fondation en briques": (100, 250),
    "rénovation de mur en béton coulé": (300, 500)  # € par mètre carré
}


### Calcul du coût de réhabilitation

In [2]:
def calculate_building_rehabilitation_cost(surface_terrain, salon_surface, chambres_surface,  salle_de_bain_surface, cuisine_surface, salle_a_manger_surface, nombre_chambres, indice_standing):
  
    surface_totale = salon_surface + salle_a_manger_surface + salle_de_bain_surface + chambres_surface + cuisine_surface
    
    ############ Define variables ##############
    surface_tuile = 0.16 #constant
    probleme_terrain = 0
    renovation_interieure = 0
    renovation_salon = 0
    renovation_chambres = 0
    renovation_salle_de_bain = 0
    renovation_cuisine = 0
    renovation_salle_a_manger = 0
    renovation_toiture = 0
    surface_toiture = 0
    surface_toiture_reparation = 0
    remplacement_toiture = 0
    remplacement_toiture_materiau = 0
    reparation_charpente = 0
    probleme_moisissure = 0
    probleme_moisissure_plafond = 0
    probleme_moisissure_sol = 0
    probleme_moisissure_gravite = 0
    probleme_electriques = 0
    probleme_norme_electriques = 0
    probleme_plomberie = 0
    probleme_plomberie_indice = 0
    probleme_hvac = 0
    probleme_hvac_indice = 0
    choix_chauffage = 0
    reparation_clim = 0
    installation_vmc = 0
    probleme_isolation = 0
    probleme_isolation_murs = 0
    probleme_isolation_murs_surface = 0
    probleme_isolation_combles = 0
    probleme_isolation_combles_surface = 0
    probleme_isolation_plancher = 0
    probleme_isolation_plancher_surface = 0
    probleme_nuisibles = 0
    probleme_rats = 0
    probleme_punaises = 0
    probleme_guepes = 0
    probleme_fourmis = 0
    probleme_nuisibles_reparation = 0
    probleme_nuisibles_surface = 0
    probleme_humidite = 0
    probleme_humidite_traitement = 0
    probleme_fondations = 0
    probleme_fondations_fissure = 0
    probleme_fondations_affaissement = 0
    remplacement_fondations_pieux = 0
    probleme_fondations_murs = 0
    probleme_fondations_murs_type = 0
    probleme_fondations_murs_surface = 0

    ############ User input data ############

    ### Input terrain ###
    print("""répondez : (oui -> 1), (non -> 0)""")
    probleme_terrain = int(input("Votre logement a-t'il des problèmes de terrain ? : "))
    renovation_interieure = int(input("Voulez-vous rénover votre intérieur ? : "))
    if renovation_interieure :
        renovation_salon = int(input("Voulez-vous rénover votre salon ? : "))
        renovation_chambres = int(input("Voulez-vous rénover vos chambres ? : "))
        renovation_salle_de_bain = int(input("Voulez-vous rénover votre salle de bain ? : "))
        renovation_cuisine = int(input("Voulez-vous rénover votre cuisine ? : "))
        renovation_salle_a_manger = int(input("Voulez-vous rénover votre salle à manger ? : "))

    ### Input toiture ### 
    renovation_toiture = int(input("Avez vous des problèmes de toiture ? : "))
    if renovation_toiture :
        surface_toiture = int(input("quelle est la surface de votre toiture ? : "))
        surface_toiture_reparation = int(input("Quelle est la surface à faire réparer ? : "))
        remplacement_toiture = int(input("Souhaitez-vous plutôt changer l'intégralité de votre toiture ? : "))
        remplacement_toiture_materiau = int(input("Quel matériau souhaitez-vous utiliser ? : \n - (0) tuiles \n - (1) ardoise \n - (2) zinc \n - (3) bac acier : "))
        reparation_charpente = int(input("devez-vous changer votre charpente ? : "))

    ### Input moisissurre ### 
    probleme_moisissure = int(input("avez-vous des problèmes de moisissure ? : "))
    if probleme_moisissure :
        probleme_moisissure_plafond = int(input("Est-ce que cela concerne le plafond ? : "))
        probleme_moisissure_sol = int(input("Est-ce que cela concerne le sol ? : "))
        probleme_moisissure_gravite = int(input("Est-ce que la moisissure occupe une pièce entière ? "))

     
    ### Input elec ###
    probleme_electriques = int(input("Est-ce que vous avez des problèmes électriques ? : "))
    if probleme_electriques :
        probleme_norme_electriques = int(input("Vos installations électriques sont-elles aux normes ? : "))
    probleme_plomberie = int(input("Est-ce que vous avez des problèmes de promberie ? : "))
    if probleme_plomberie :
        probleme_plomberie_indice = int(input("Sont-ils graves : Non -> 0 , Oui -> 1 ? : "))
    probleme_hvac = int(input("Avez-vous des problèmes de ventilation, chauffage ? : "))

    ### Input HVAC ###
    if probleme_hvac :
        probleme_hvac_indice = int(input("Quelle gamme de chaudière souhaitez-vous ? : de 0 à 3 : "))
        choix_chauffage = int(input("Préférez-vous une chaudière (0) ou une pompe à chaleur (1) ? : "))
        reparation_clim = int(input("Devez-vous réparer/installer votre climatisation ? : "))
        installation_vmc = int(input("Souhaitez-vous installer une VMC ? : "))

    ###Input isolation ###
    probleme_isolation = int(input("Avez-vous des problemes d'isolation ? : "))
    if probleme_isolation :
        probleme_isolation_murs = int(input("Souhaitez-vous isoler vos murs ? : "))
        probleme_isolation_murs_surface = int(input("Quelle surface devez-vous isoler ? : "))
        probleme_isolation_combles = int(input("Souhaitez-vous isoler vos combles ? : "))
        probleme_isolation_combles_surface = int(input("Quelle surface devez-vous isoler ? : "))
        probleme_isolation_plancher = int(input("Souhaitez-vous isoler votre plancher ? : "))
        probleme_isolation_plancher_surface = int(input("Quelle surface devez-vous isoler ? : "))   
    
    ### Input nuisibles ###
    probleme_nuisibles = int(input("Avez-vous des problèmes de nuisibles ? : "))
    if probleme_nuisibles :
        probleme_rats = int(input("Etes-vous infesté par les rats ? : "))
        probleme_punaises = int(input("Etes-vous infesté par les punaises de lit : "))
        probleme_guepes = int(input("Etes-vous infesté par les guèpes ? : "))
        probleme_fourmis = int(input("Etes-vous infesté par les fourmis ? : "))
        probleme_nuisibles_reparation = int(input("Avez-vous des dommages causés par ces nuisibles ? : "))
        probleme_nuisibles_surface = int(input("Quelle est la surface concernée ? : "))
        probleme_humidite = int(input("Avez-vous des problèmes d'humidité ? : "))
        probleme_humidite_traitement = int(input("Vous souhaitez utiliser la méthode : \n -(0) traitement par injection \n - (1) traitement par ventilation \n - (2) traitement par drainage : "))

    ### Input fondations ###
    probleme_fondations = int(input("Vos fondations sont-elles endommagées ? : "))
    if probleme_fondations :
        probleme_fondations_fissure = int(input("Vos fondations sont-elles fissurées ? : "))
        probleme_fondations_affaissement = int(input("Vos fondations se sont-elles affaissées ? : "))
        remplacement_fondations_pieux = int(input("Devez-vous changer les pieux de vos fondations ? : "))
        probleme_fondations_murs = int(input("Vos murs de fondations sont-ils endommagés ? : "))
        if probleme_fondations_murs :
            probleme_fondations_murs_surface = int(input("Quelle est la surface du mur à réparer ? : "))
            probleme_fondations_murs_type = int(input("Choisissez le type de mur à rénover: \n - (0) Mur en bloc de béton \n - (1) Mur en briques \n - (2) Mur en béton coulé : ")) 



    ############ Coût des problèmes de terrain ############
    geotechnical_investigation_cost = 0
    if probleme_terrain :
        geotechnical_investigation_cost += surface_terrain * geotechnical_investigation_prices["étude topographique"]
        geotechnical_investigation_cost += geotechnical_investigation_prices["rapport ingénieur"]

    print("Terrain : ", geotechnical_investigation_cost)

    ############ Coût démoliton pièces, murs, etc ############
    destruction_cost = 0
    
    if renovation_salon : 
        destruction_cost += demolition_prices["porte"] + demolition_prices["fenêtre"] + demolition_prices["cloison en plâtre"] + demolition_prices["murs porteurs"]*math.sqrt(salon_surface)* 2 #utilisation racine pour calculer longueur du mur + hauteur par défaut de 2m
    if renovation_chambres :
        destruction_cost += demolition_prices["lampe incandescente"] + demolition_prices["porte"] + demolition_prices["fenêtre"]

    if renovation_salle_de_bain :
        destruction_cost += demolition_prices["lavabo"] + demolition_prices["baignoire"] + demolition_prices["douche"] + demolition_prices["porte"] + demolition_prices["fenêtre"]

    if renovation_cuisine :
        destruction_cost += demolition_prices["lavabo"] + demolition_prices["lampe incandescente"] + demolition_prices["lave-vaisselle"] 

    if renovation_salle_a_manger :
        demolition_prices["lampe incandescente"] + demolition_prices["porte"] + demolition_prices["cloison en plâtre"]* math.sqrt(salle_a_manger_surface) * 2 #utilisation racine pour calculer longueur du mur + hauteur par défaut de 2m

    if renovation_toiture :
        if reparation_charpente :
            destruction_cost += demolition_prices["charpente"]*surface_toiture
        destruction_cost += demolition_prices["tuiles"]*surface_toiture

    print("Demolition pièces : ", destruction_cost)

    ############ Coût des problèmes liés à la moisissure ############
    mold_cost = 0
    if probleme_moisissure :
        mold_cost += mold_prices["inspection de la zone"] + mold_prices["réduction de la moisissure"] 
    if probleme_moisissure_gravite :
        mold_cost += mold_prices["enlèvement de zones contaminées par la moisissure"] 
    if probleme_moisissure_sol : 
        mold_cost += mold_demolition_prices["sol"] * salle_de_bain_surface
    if probleme_moisissure_plafond :
        mold_cost += mold_demolition_prices["plafond"] * salle_de_bain_surface

    print("Moisissure : ", mold_cost)

    ############ Coût des problèmes électriques ############

    electrical_cost = 0
    if probleme_electriques :
        electrical_cost += electrical_replacement_prices["remplacement du panneau électrique"][indice_standing] + electrical_replacement_prices["réparation ou remplacement des fils et des câbles"][indice_standing] + electrical_replacement_prices["installation de nouveaux circuits électriques"][indice_standing] + electrical_replacement_prices["installation de systèmes d'éclairage"][indice_standing]

        if not probleme_norme_electriques : 
            electrical_cost += electrical_upgrade_prices["mise à niveau des prises de courant et des interrupteurs"][indice_standing] + electrical_upgrade_prices["mise à jour des systèmes de sécurité"][indice_standing]

    print("Elec : ", electrical_cost)

    ############ Réparation plomberie ############

    plumbing_cost = 0
    if renovation_salle_de_bain : 
        plumbing_cost += bathroom_prices["lavabo"] + bathroom_prices["baignoire/douche"] + bathroom_prices["toilette"][indice_standing] + bathroom_prices["robinet de douche"][indice_standing] + bathroom_prices["robinet de lavabo"][indice_standing]  



    if probleme_plomberie :
        plumbing_cost += plumbing_repair_prices["remplacement d'un tuyau"][indice_standing] + plumbing_repair_prices["remplacement d'un robinet"][indice_standing] + plumbing_repair_prices["débouchage d'un évier"][indice_standing] + plumbing_repair_prices["remplacement d'un évier"][indice_standing] + plumbing_repair_prices["réparation d'une fuite"][probleme_plomberie_indice]

    print("Plomberie : ", plumbing_cost)

    ############## HVAC ##############
    hvac_cost = 0
    if probleme_hvac :
        hvac_cost += hvac_replacement_repair_prices["ventilation + conduits"]  
        if choix_chauffage : 
            hvac_cost += hvac_replacement_repair_prices["chaudière"][probleme_hvac_indice]
        else :
            hvac_cost += hvac_replacement_repair_prices["installation d'un nouveau système de chauffage (pompe à chaleur)"][indice_standing]
        if reparation_clim :
            hvac_cost += hvac_replacement_repair_prices["réparation ou remplacement de la climatisation"][indice_standing]
        if installation_vmc :
            hvac_cost += hvac_replacement_repair_prices["installation d'un système de ventilation mécanique contrôlée (VMC)"][indice_standing]

    print("HVAC : ", hvac_cost)

    ######## Coût isolation ########
    insulation_cost = 0

    if probleme_isolation_combles :
        insulation_cost += insulation_prices["isolation des combles"][indice_standing] * probleme_isolation_combles_surface
    if probleme_isolation_murs :
        insulation_cost += insulation_prices["isolation des murs"][indice_standing] * probleme_isolation_murs_surface + 2*insulation_prices["isolation des portes"][indice_standing] + 2*insulation_prices["installation de fenêtres isolantes"][indice_standing]
    if probleme_isolation_plancher :
        insulation_cost += insulation_prices["isolation des planchers"][indice_standing] * probleme_isolation_plancher_surface


    print("Isolation : ", insulation_cost)
    

    ######### Pest #########
    pest_cost = 0 
    if probleme_guepes :
        pest_cost += pest_control_prices["élimination des guêpes et des nids de frelons"][indice_standing]
        if probleme_nuisibles_reparation :
            pest_cost += pest_damage_repair_prices["réparation des dommages causés par les insectes"][indice_standing] + probleme_nuisibles_surface * pest_damage_repair_prices["nettoyage et désinfection de la zone touchée"][indice_standing]
    if probleme_punaises :
        pest_cost += pest_control_prices["élimination de punaises de lit"][indice_standing]
    if probleme_fourmis : 
        pest_cost += pest_control_prices["élimination des fourmis"][indice_standing]
    if probleme_rats :
        pest_cost += pest_control_prices["élimination de souris et de rats"][indice_standing] 
        if probleme_nuisibles_reparation :
            pest_cost += pest_damage_repair_prices["réparation des dommages causés par les rongeurs"][indice_standing] + probleme_nuisibles_surface * pest_damage_repair_prices["nettoyage et désinfection de la zone touchée"][indice_standing]

    print("Nuisibles : ", pest_cost)


    ########## Problèmes d'humidité #########
    humidity_treatment_cost = 0
    if probleme_humidite :
        humidity_treatment_cost += humidity_detection_treatment_prices["diagnostic de l'humidité"] * round(math.sqrt(salle_de_bain_surface))
        if probleme_humidite_traitement == 0 :
            humidity_treatment_cost += humidity_detection_treatment_prices["traitement de l'humidité par injection"][indice_standing]
        if probleme_humidite_traitement == 1 :
            humidity_treatment_cost += humidity_detection_treatment_prices["traitement de l'humidité par ventilation"][indice_standing]
        if probleme_humidite_traitement == 2 :
            humidity_treatment_cost += humidity_detection_treatment_prices["traitement de l'humidité par drainage"][indice_standing]

    print("Humidité : ", humidity_treatment_cost)

    ########## Problèmes toiture ########
    roof_repair_cost = 0
    if not remplacement_toiture and renovation_toiture :
        roof_repair_cost += int(surface_tuile * float(surface_toiture_reparation)) * roof_repair_prices["remplacement de tuiles"][indice_standing] + roof_repair_prices["réparation de fuite"][indice_standing] + surface_toiture_reparation * roof_repair_prices["remplacement du revêtement de toit"][indice_standing] + roof_repair_prices["nettoyage de la toiture"][indice_standing]
    if remplacement_toiture :
        roof_repair_cost += roof_repair_prices["remplacement du revêtement de toit"][indice_standing]*surface_toiture
        if remplacement_toiture_materiau == 0 :
            roof_repair_cost += surface_toiture * roof_replacement_prices["toiture en tuiles"][indice_standing]
        if remplacement_toiture_materiau == 1 :
            roof_repair_cost += surface_toiture * roof_replacement_prices["toiture en ardoise"][indice_standing]
        if remplacement_toiture_materiau == 2 :
            roof_repair_cost += surface_toiture * roof_replacement_prices["toiture en zinc"][indice_standing]
        if remplacement_toiture_materiau == 3 :
            roof_repair_cost += surface_toiture * roof_replacement_prices["toiture en bac acier"][indice_standing]

    print("Toiture: ", roof_repair_cost)

    ###### Coût problèmes fondations ######
    foundation_repair_cost = 0
    if probleme_fondations :
        if probleme_fondations_fissure :
            foundation_repair_cost += foundation_crack_repair_prices["réparation de fissures - injection de résine époxy"][indice_standing] + foundation_crack_repair_prices["ancrage de la fondation"][indice_standing]
        if probleme_fondations_affaissement :
            foundation_repair_cost += foundation_subsidence_repair_prices["injection de mousse polyuréthane"][indice_standing]
            if remplacement_fondations_pieux :
                foundation_repair_cost += foundation_subsidence_repair_prices["remplacement des pieux"][indice_standing]

        if probleme_fondations_murs :
            if probleme_fondations_murs_type == 0 :
                foundation_repair_cost += foundation_wall_repair_prices["rénovation de mur en blocs de béton"][indice_standing] * probleme_fondations_murs_surface
            if probleme_fondations_murs_type == 1 :
                foundation_repair_cost += foundation_wall_repair_prices["réparation de murs de fondation en briques"][indice_standing] * probleme_fondations_murs_surface
            if probleme_fondations_murs_type == 2 :
                foundation_repair_cost += foundation_wall_repair_prices["rénovation de mur en béton coulé"][indice_standing] * probleme_fondations_murs_surface

    print("Fondations : ", foundation_repair_cost)

    ###### Somme coût total ######
    total_cost = geotechnical_investigation_cost + destruction_cost + mold_cost + electrical_cost + plumbing_cost + hvac_cost + insulation_cost + pest_cost + roof_repair_cost + foundation_repair_cost

    print("Total : ")
    return total_cost

### Test de la fonction 

In [3]:
print(calculate_building_rehabilitation_cost(2000, 10, 10, 10, 10, 10, 0, 1))

répondez : (oui -> 1), (non -> 0)
